In [1]:
import os
import sys
import urllib.request
import json
import datetime
import pandas as pd


/tmp/ipykernel_21237/4190591205.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
client_id = "zksStlRfc9DFTDwYr8rp"
client_secret = "8AHTEDR12X"

In [3]:
def gen_search_url(api_node, search_text, start_num, disp_num):
    base = "https://openapi.naver.com/v1/search"
    node = "/" + api_node + ".json"
    param_query = "?query=" + urllib.parse.quote(search_text)
    param_start = "&start=" + str(start_num)
    param_disp = "&display=" + str(disp_num)

    return base + node + param_query + param_start + param_disp

gen_search_url('shop', 'TEST', 10, 3)

'https://openapi.naver.com/v1/search/shop.json?query=TEST&start=10&display=3'

In [4]:
def get_result_onpage(url):
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-naver-Client-Secret", client_secret)

    response = urllib.request.urlopen(request)

    print("[%s] Url Request Success" % datetime.datetime.now())

    return json.loads(response.read().decode('utf-8'))

In [5]:
url = gen_search_url('shop', '몰스킨', 1, 5)
one_result = get_result_onpage(url)
one_result

[2024-02-23 16:32:31.319019] Url Request Success


{'lastBuildDate': 'Fri, 23 Feb 2024 16:32:31 +0900',
 'total': 47006,
 'start': 1,
 'display': 5,
 'items': [{'title': '[<b>몰스킨</b>] 2024년 클래식 다이어리 (12개월) (데일리, 위클리, 먼슬리)',
   'link': 'https://search.shopping.naver.com/gate.nhn?id=42344941884',
   'image': 'https://shopping-phinf.pstatic.net/main_4234494/42344941884.jpg',
   'lprice': '16200',
   'hprice': '',
   'mallName': '몰스킨공식온라인스토어',
   'productId': '42344941884',
   'productType': '2',
   'brand': '몰스킨',
   'maker': '',
   'category1': '생활/건강',
   'category2': '문구/사무용품',
   'category3': '다이어리/플래너',
   'category4': '다이어리'},
  {'title': '올젠 <b>몰스킨</b> 자켓 ZOD4KG1311',
   'link': 'https://search.shopping.naver.com/gate.nhn?id=42816582647',
   'image': 'https://shopping-phinf.pstatic.net/main_4281658/42816582647.20240215125021.jpg',
   'lprice': '72110',
   'hprice': '',
   'mallName': '네이버',
   'productId': '42816582647',
   'productType': '1',
   'brand': '올젠',
   'maker': '',
   'category1': '패션의류',
   'category2': '남성의류',
   'cat

In [6]:
one_result['items'][0]['title']

'[<b>몰스킨</b>] 2024년 클래식 다이어리 (12개월) (데일리, 위클리, 먼슬리)'

In [7]:
one_result['items'][0]['lprice']

'16200'

In [8]:
one_result['items'][0]['mallName']

'몰스킨공식온라인스토어'

In [9]:
def get_fields(json_data):
    title = [each['title'] for each in json_data['items']]
    link = [each['link'] for each in json_data['items']]
    lprice = [each['lprice'] for each in json_data['items']]
    hprice = [each['hprice'] for each in json_data['items']]
    mall_name = [each['mallName'] for each in json_data['items']]

    result_pd = pd.DataFrame({'title':title, 'lprice':lprice,
                              'hprice':hprice, 'link':link,
                              'mall':mall_name},
                              columns=['title', 'lprice', 'hprice', 'link', 'mall'])
    
    return result_pd

In [10]:
get_fields(one_result)

,title,lprice,hprice,link,mall
0,"[<b>몰스킨</b>] 2024년 클래식 다이어리 (12개월) (데일리, 위클리, ...",16200,,https://search.shopping.naver.com/gate.nhn?id=...,몰스킨공식온라인스토어
1,올젠 <b>몰스킨</b> 자켓 ZOD4KG1311,72110,,https://search.shopping.naver.com/gate.nhn?id=...,네이버
2,<b>몰스킨</b> 2024 다이어리 클래식 소프트커버 데일리 라지 2024년 24년,23400,,https://search.shopping.naver.com/gate.nhn?id=...,베스트펜
3,르몽생미셸 지니언 워크자켓 블루 <b>몰스킨</b> 프렌치 워크자켓,378000,,https://search.shopping.naver.com/gate.nhn?id=...,스프링스트릿 마켓
4,르몽생미셸 프렌치 <b>몰스킨</b> 워크자켓 지니언 블루,154000,,https://search.shopping.naver.com/gate.nhn?id=...,11번가


In [11]:
def delete_tag(input_str):
    input_str = input_str.replace("<b>", "")
    input_str = input_str.replace("</b>", "")

    return input_str

In [12]:
def get_fields(json_data):
    title = [delete_tag(each['title']) for each in json_data['items']]
    link = [each['link'] for each in json_data['items']]
    lprice = [each['lprice'] for each in json_data['items']]
    hprice = [each['hprice'] for each in json_data['items']]
    mall_name = [each['mallName'] for each in json_data['items']]

    result_pd = pd.DataFrame({'title':title, 'lprice':lprice,
                              'hprice':hprice, 'link':link,
                              'mall':mall_name},
                              columns=['title', 'lprice', 'hprice', 'link', 'mall'])
    
    return result_pd

In [13]:
url = gen_search_url('shop', '몰스킨', 1, 5)
json_result = get_result_onpage(url)
pd_result = get_fields(json_result)
pd_result

[2024-02-23 16:32:31.427108] Url Request Success


,title,lprice,hprice,link,mall
0,"[몰스킨] 2024년 클래식 다이어리 (12개월) (데일리, 위클리, 먼슬리)",16200,,https://search.shopping.naver.com/gate.nhn?id=...,몰스킨공식온라인스토어
1,올젠 몰스킨 자켓 ZOD4KG1311,72110,,https://search.shopping.naver.com/gate.nhn?id=...,네이버
2,몰스킨 2024 다이어리 클래식 소프트커버 데일리 라지 2024년 24년,23400,,https://search.shopping.naver.com/gate.nhn?id=...,베스트펜
3,르몽생미셸 지니언 워크자켓 블루 몰스킨 프렌치 워크자켓,378000,,https://search.shopping.naver.com/gate.nhn?id=...,스프링스트릿 마켓
4,르몽생미셸 프렌치 몰스킨 워크자켓 지니언 블루,154000,,https://search.shopping.naver.com/gate.nhn?id=...,11번가


In [14]:
for n in range(1,1000,100):
    print(n)

1
101
201
301
401
501
601
701
801
901


In [15]:
result_mol = []

for n in range(1,1000,100):
    url = gen_search_url('shop', '"몰스킨"', n, 100)
    json_result = get_result_onpage(url)
    pd_result = get_fields(json_result)

    result_mol.append(pd_result)

result_mol = pd.concat(result_mol)

[2024-02-23 16:32:31.489144] Url Request Success
[2024-02-23 16:32:31.546298] Url Request Success


[2024-02-23 16:32:31.820493] Url Request Success
[2024-02-23 16:32:32.099338] Url Request Success
[2024-02-23 16:32:32.164037] Url Request Success
[2024-02-23 16:32:32.223649] Url Request Success
[2024-02-23 16:32:32.522957] Url Request Success
[2024-02-23 16:32:32.786164] Url Request Success
[2024-02-23 16:32:32.865867] Url Request Success
[2024-02-23 16:32:32.937250] Url Request Success


In [16]:
result_mol.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 99
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   1000 non-null   object
 1   lprice  1000 non-null   object
 2   hprice  1000 non-null   object
 3   link    1000 non-null   object
 4   mall    1000 non-null   object
dtypes: object(5)
memory usage: 46.9+ KB


In [17]:
result_mol.reset_index(drop=True, inplace=True)
result_mol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   1000 non-null   object
 1   lprice  1000 non-null   object
 2   hprice  1000 non-null   object
 3   link    1000 non-null   object
 4   mall    1000 non-null   object
dtypes: object(5)
memory usage: 39.2+ KB


In [18]:
result_mol.head()

,title,lprice,hprice,link,mall
0,"[몰스킨] 2024년 클래식 다이어리 (12개월) (데일리, 위클리, 먼슬리)",16200,,https://search.shopping.naver.com/gate.nhn?id=...,몰스킨공식온라인스토어
1,몰스킨 2024 다이어리 클래식 소프트커버 데일리 라지 2024년 24년,23400,,https://search.shopping.naver.com/gate.nhn?id=...,베스트펜
2,스타벅스 2024년 몰스킨 다이어리 데일리 플래너 일기장 노트 스벅 카페 윈터 e프리퀀시,28000,,https://search.shopping.naver.com/gate.nhn?id=...,유니스스토어
3,몰스킨 2024 위클리 다이어리 라지 상품선택,25200,,https://search.shopping.naver.com/gate.nhn?id=...,네이버
4,몰스킨 2024년 위클리 하드커버 라지 다이어리,26640,,https://search.shopping.naver.com/gate.nhn?id=...,펜카페스토어


In [19]:
result_mol['lprice'] = pd.to_numeric(result_mol['lprice'], errors='coerce')
result_mol['hprice'] = pd.to_numeric(result_mol['hprice'], errors='coerce')
result_mol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   title   1000 non-null   object 
 1   lprice  1000 non-null   int64  
 2   hprice  0 non-null      float64
 3   link    1000 non-null   object 
 4   mall    1000 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 39.2+ KB


In [20]:
writer = pd.ExcelWriter("../data/molskin_diary_in_naver_shop.xlsx",
                        engine='xlsxwriter')
result_mol.to_excel(writer, sheet_name='Sheet1')

workbook = writer.book
worksheet = writer.sheets['Sheet1']
worksheet.set_column('A:A', 4)
worksheet.set_column('B:B', 65)
worksheet.set_column('C:C', 10)
worksheet.set_column('D:D', 10)
worksheet.set_column('E:E', 50)
worksheet.set_column('F:F', 15)

worksheet.conditional_format('C2:C1001', {'type': '3_color_scale'})

writer.close()

In [23]:
pd.__dir__()

['__name__',
 '__doc__',
 '__package__',
 '__loader__',
 '__spec__',
 '__path__',
 '__file__',
 '__cached__',
 '__builtins__',
 'annotations',
 '__docformat__',
 'util',
 'compat',
 '_is_numpy_dev',
 '_typing',
 '_config',
 'get_option',
 'set_option',
 'reset_option',
 'describe_option',
 'option_context',
 'options',
 'core',
 'pandas',
 '_pandas_parser_CAPI',
 '_pandas_datetime_CAPI',
 '_libs',
 'errors',
 'io',
 'tseries',
 'arrays',
 'plotting',
 'ArrowDtype',
 'Int8Dtype',
 'Int16Dtype',
 'Int32Dtype',
 'Int64Dtype',
 'UInt8Dtype',
 'UInt16Dtype',
 'UInt32Dtype',
 'UInt64Dtype',
 'Float32Dtype',
 'Float64Dtype',
 'CategoricalDtype',
 'PeriodDtype',
 'IntervalDtype',
 'DatetimeTZDtype',
 'StringDtype',
 'BooleanDtype',
 'NA',
 'isna',
 'isnull',
 'notna',
 'notnull',
 'Index',
 'CategoricalIndex',
 'RangeIndex',
 'MultiIndex',
 'IntervalIndex',
 'TimedeltaIndex',
 'DatetimeIndex',
 'PeriodIndex',
 'IndexSlice',
 'NaT',
 'Period',
 'period_range',
 'Timedelta',
 'timedelta_range',
